In [1]:
# ignore warning
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None


from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,classification_report,f1_score
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, cross_val_predict

# from catboost import CatBoostClassifier
# import lightgbm as lgb


import plotly.express as px
import plotly.graph_objects as go

sns.set()

In [2]:
from catboost import CatBoostClassifier


In [67]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13314 entries, 0 to 13313
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Feature 1   13305 non-null  float64
 1   Feature 2   13309 non-null  float64
 2   Feature 3   13308 non-null  float64
 3   Feature 4   13309 non-null  float64
 4   Feature 5   13310 non-null  float64
 5   Feature 6   13294 non-null  float64
 6   Feature 7   13311 non-null  float64
 7   Feature 8   13313 non-null  float64
 8   Feature 9   13306 non-null  float64
 9   Feature 10  13312 non-null  object 
 10  Feature 11  13308 non-null  float64
 11  Feature 12  13310 non-null  float64
 12  Class       13314 non-null  object 
dtypes: float64(11), object(2)
memory usage: 1.3+ MB


In [69]:
train[train["Feature 1"].isnull()]

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Class
1651,NaN,32.2038,3.3617,0.413610,0.3471,42.2975,95.9232,-14.0660,10.513333,A,136.749333,4.8572,a
1652,NaN,22.6300,3.2064,0.383350,0.3035,9.1080,10.5867,7.8929,10.408067,H,96.321333,4.4796,a
1653,NaN,30.7476,3.3014,0.372470,0.2761,30.7541,-12.5821,-4.5224,17.721467,H,73.738000,3.8955,a
1654,NaN,32.6498,3.4276,0.332180,0.3095,16.3124,23.0992,-14.0440,0.173000,B,150.116000,2.0552,a
1655,NaN,68.7716,3.8367,0.151725,0.1397,-187.8540,89.6046,-9.6529,1.238000,K,153.973333,2.9517,b
12713,NaN,38.4834,3.8933,0.195585,0.1855,-8.0591,-25.5224,9.5210,7.680333,L,134.644667,4.9986,a
12996,NaN,21.5362,3.5145,0.390320,0.3209,14.5489,29.3867,9.8552,38.110667,H,140.695333,4.2419,b
13274,NaN,92.7854,3.9114,0.225505,0.1739,-200.1309,-161.5741,36.3618,38.897600,G,195.062200,0.7877,b
13282,NaN,53.7036,3.6294,0.166090,0.1576,32.3797,30.4534,15.2517,58.447867,K,84.236667,3.7051,a


In [70]:
train.head()

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Class
0,22.1764,34.6044,3.5728,0.205785,0.1786,-52.7420,10.1133,-9.4946,4.643000,L,146.933333,0.6187,a
1,16.7913,22.0692,3.4647,0.365585,0.2959,39.1995,26.6283,6.3241,18.010000,H,95.957333,4.9524,b
2,20.4557,23.6334,NaN,0.415820,0.3562,8.0176,32.4742,5.4401,6.666267,A,78.585400,2.9143,b
3,8.9238,20.9858,3.0242,0.510935,0.4037,21.8677,8.6185,-2.2203,29.190867,R,43.343333,6.3982,a
4,20.3489,61.4794,4.2051,0.134980,NaN,40.9395,22.9621,8.9797,5.257467,F,84.718667,3.9418,a


In [71]:
train["Feature 1"].median()

18.6237

In [72]:
train.shape, test.shape

((13314, 13), (5706, 13))

In [73]:
X_test=test.copy()

In [74]:
dataset = pd.concat([train, test])

In [75]:
dataset["Feature 10"].value_counts()

K    1905
B    1905
F    1903
N    1902
G    1902
R    1901
A    1901
L    1900
M    1900
H    1899
Name: Feature 10, dtype: int64

In [76]:
len(list)

19020

In [77]:
dataset.shape

(19020, 14)

In [78]:
dataset.columns

Index(['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5',
       'Feature 6', 'Feature 7', 'Feature 8', 'Feature 9', 'Feature 10',
       'Feature 11', 'Feature 12', 'Class', 'ID'],
      dtype='object')

In [79]:
dataset["Feature 10"].fillna("F", inplace = True)

In [80]:
dataset["Feature 10"].isnull().sum()

0

In [81]:
for i in dataset.columns[:-1]:
    if i != "Feature 10" and "Class":
        print(i)
        dataset[i].fillna(dataset[i].mean(), inplace = True)

Feature 1
Feature 2
Feature 3
Feature 4
Feature 5
Feature 6
Feature 7
Feature 8
Feature 9
Feature 11
Feature 12
Class


TypeError: can only concatenate str (not "int") to str

In [82]:
dataset.isnull().sum()

Feature 1         0
Feature 2         0
Feature 3         0
Feature 4         0
Feature 5         0
Feature 6         0
Feature 7         0
Feature 8         0
Feature 9         0
Feature 10        0
Feature 11        0
Feature 12        0
Class          5706
ID            13314
dtype: int64

In [83]:
list=[]
for i in dataset["Feature 1"]:
    if i <50:
        list.append("low")
    elif 100>i >50:
        list.append("medium")
    elif 100<i<168:
        list.append("high")
dataset["EFFECT"]=list

In [84]:
dataset["Feature 10"] = pd.Categorical(dataset["Feature 10"]).codes

In [85]:
#dataset["EFFECT"]=list
dataset["EFFECT"] = pd.Categorical(dataset["EFFECT"]).codes

In [86]:
dataset

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Class,ID,EFFECT
0,22.17640,34.6044,3.572800,0.205785,0.178600,-52.7420,10.1133,-9.4946,4.643000,6,146.933333,0.6187,a,NaN,1
1,16.79130,22.0692,3.464700,0.365585,0.295900,39.1995,26.6283,6.3241,18.010000,4,95.957333,4.9524,b,NaN,1
2,20.45570,23.6334,3.580468,0.415820,0.356200,8.0176,32.4742,5.4401,6.666267,0,78.585400,2.9143,b,NaN,1
3,8.92380,20.9858,3.024200,0.510935,0.403700,21.8677,8.6185,-2.2203,29.190867,9,43.343333,6.3982,a,NaN,1
4,20.34890,61.4794,4.205100,0.134980,0.264671,40.9395,22.9621,8.9797,5.257467,2,84.718667,3.9418,a,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5701,51.19565,138.8176,3.500000,0.260610,0.203300,-72.3991,-109.6677,-19.5580,19.502267,7,267.412133,3.7743,NaN,16632.0,2
5702,14.36995,12.5746,3.063900,0.548760,0.460200,30.3165,18.5648,-4.3903,13.167267,9,118.628000,0.1745,NaN,11032.0,1
5703,9.57665,20.1472,3.063900,0.462145,0.329100,26.6041,2.4195,4.5020,46.480400,0,148.561333,6.9215,NaN,15068.0,1
5704,11.95130,35.7210,3.245500,0.325890,0.243300,12.0952,15.7668,-17.3896,42.305467,1,88.008667,5.3772,NaN,10456.0,1


In [131]:
X=dataset[  ['Feature 1', 'Feature 2', 'Feature 3', 'Feature 7',  'Feature 9', 'Feature 11']]
y=dataset['Class']

In [132]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [133]:
X_train = X[:-5706]
y_train = y[:-5706]

In [134]:
pd.DataFrame(X_train).isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

In [135]:
# X_train = X[:10000]
# y_train = y[:10000]

In [136]:
#9000:4314

In [137]:
dataset.shape

(19020, 15)

In [138]:
(19020-5706)-5706

7608

In [139]:
pd.DataFrame(y_train).isnull().sum()

Class    0
dtype: int64

In [140]:
X_train.shape

(13314, 6)

In [141]:
y_train=pd.DataFrame(y_train,columns=["Class"])

In [142]:
y_train=pd.DataFrame(y_train,columns=["Class"])
y_train.loc[y_train['Class'] == "a", 'Class',] = 0
y_train.loc[y_train['Class'] == "b", 'Class',] = 1

In [143]:
y_train['Class']=y_train['Class'].astype(int)

In [144]:
y_train=y_train.values

In [145]:
y_train.shape

(13314, 1)

In [146]:
X_train

array([[-2.09971512e-01, -2.65816437e-01, -1.44045784e-02,
         5.03987024e-02, -7.92279577e-01,  3.55784663e-01],
       [-4.64300809e-01, -6.07588425e-01, -2.17476397e-01,
         3.74248109e-01, -2.39140623e-02, -6.67708151e-01],
       [-2.91237305e-01, -5.64940542e-01,  4.08780589e-14,
         4.88882759e-01, -6.75977615e-01, -1.01650070e+00],
       ...,
       [-1.44931013e-01, -7.72463759e-02,  5.36199955e-01,
         4.26617000e-01, -5.65397054e-01, -2.10453260e-01],
       [-6.67327007e-02, -3.30379912e-01,  5.26243612e-01,
        -8.03267549e-01, -6.87795970e-01, -9.02713687e-01],
       [ 4.17436962e-02, -5.63533669e-01,  1.04132256e-01,
         5.95218707e-01,  8.84765676e-01, -1.58444658e+00]])

In [147]:
from sklearn.metrics import accuracy_score

no_splits = 10
kfold = KFold(n_splits=no_splits, shuffle=True, random_state=1)
counter = 0
models = {}
scores = []
model_s=[]
for train, test in kfold.split(X_train):
    counter += 1
    print(train.shape, test.shape)
    train_X, test_X = X_train[train], X_train[test]
    train_y, test_y = y_train[train], y_train[test]
    print(train_y.shape, train_y.shape)
    print("end")
#   fit model
    model =CatBoostClassifier()
    #print(train_X,train_y)
    model.fit(train_X, train_y)
    models[f"model{str(counter)}"] = model
#   evaluate model
    yhat = model.predict(test_X)
    acc = accuracy_score(test_y, yhat)
    model_s.append(model)
    scores.append(acc)

(11982,) (1332,)
(11982, 1) (11982, 1)
end
Learning rate set to 0.029749
0:	learn: 0.6750020	total: 8.02ms	remaining: 8.01s
1:	learn: 0.6568807	total: 14.5ms	remaining: 7.24s
2:	learn: 0.6399807	total: 20.7ms	remaining: 6.88s
3:	learn: 0.6249378	total: 27ms	remaining: 6.72s
4:	learn: 0.6100266	total: 35.3ms	remaining: 7.02s
5:	learn: 0.5973538	total: 47.4ms	remaining: 7.85s
6:	learn: 0.5847355	total: 54.1ms	remaining: 7.67s
7:	learn: 0.5729342	total: 62ms	remaining: 7.69s
8:	learn: 0.5612569	total: 68.2ms	remaining: 7.51s
9:	learn: 0.5513334	total: 75.1ms	remaining: 7.43s
10:	learn: 0.5415206	total: 81.4ms	remaining: 7.32s
11:	learn: 0.5313649	total: 88ms	remaining: 7.25s
12:	learn: 0.5222824	total: 94.5ms	remaining: 7.18s
13:	learn: 0.5140363	total: 101ms	remaining: 7.09s
14:	learn: 0.5070591	total: 107ms	remaining: 7.05s
15:	learn: 0.5000321	total: 114ms	remaining: 6.99s
16:	learn: 0.4935576	total: 120ms	remaining: 6.93s
17:	learn: 0.4873579	total: 126ms	remaining: 6.89s
18:	learn: 0

165:	learn: 0.3168439	total: 1.3s	remaining: 6.53s
166:	learn: 0.3165807	total: 1.31s	remaining: 6.52s
167:	learn: 0.3161974	total: 1.31s	remaining: 6.51s
168:	learn: 0.3159473	total: 1.32s	remaining: 6.5s
169:	learn: 0.3156251	total: 1.33s	remaining: 6.5s
170:	learn: 0.3153790	total: 1.34s	remaining: 6.49s
171:	learn: 0.3149792	total: 1.34s	remaining: 6.48s
172:	learn: 0.3147983	total: 1.35s	remaining: 6.47s
173:	learn: 0.3146119	total: 1.36s	remaining: 6.46s
174:	learn: 0.3144301	total: 1.37s	remaining: 6.45s
175:	learn: 0.3141665	total: 1.38s	remaining: 6.45s
176:	learn: 0.3138420	total: 1.39s	remaining: 6.44s
177:	learn: 0.3135854	total: 1.39s	remaining: 6.43s
178:	learn: 0.3133169	total: 1.4s	remaining: 6.43s
179:	learn: 0.3130081	total: 1.41s	remaining: 6.42s
180:	learn: 0.3126553	total: 1.42s	remaining: 6.41s
181:	learn: 0.3123716	total: 1.43s	remaining: 6.41s
182:	learn: 0.3120564	total: 1.43s	remaining: 6.4s
183:	learn: 0.3118062	total: 1.44s	remaining: 6.39s
184:	learn: 0.311

330:	learn: 0.2850295	total: 2.7s	remaining: 5.47s
331:	learn: 0.2848854	total: 2.73s	remaining: 5.49s
332:	learn: 0.2847702	total: 2.74s	remaining: 5.49s
333:	learn: 0.2846400	total: 2.77s	remaining: 5.52s
334:	learn: 0.2845368	total: 2.78s	remaining: 5.53s
335:	learn: 0.2844078	total: 2.82s	remaining: 5.57s
336:	learn: 0.2842935	total: 2.83s	remaining: 5.58s
337:	learn: 0.2841413	total: 2.86s	remaining: 5.61s
338:	learn: 0.2838931	total: 2.88s	remaining: 5.62s
339:	learn: 0.2837481	total: 2.89s	remaining: 5.62s
340:	learn: 0.2835760	total: 2.91s	remaining: 5.62s
341:	learn: 0.2834172	total: 2.92s	remaining: 5.61s
342:	learn: 0.2833049	total: 2.93s	remaining: 5.61s
343:	learn: 0.2831380	total: 2.94s	remaining: 5.6s
344:	learn: 0.2829623	total: 2.94s	remaining: 5.59s
345:	learn: 0.2828326	total: 2.95s	remaining: 5.58s
346:	learn: 0.2826937	total: 2.96s	remaining: 5.57s
347:	learn: 0.2825645	total: 2.97s	remaining: 5.56s
348:	learn: 0.2824408	total: 2.97s	remaining: 5.55s
349:	learn: 0.

497:	learn: 0.2631738	total: 4.15s	remaining: 4.18s
498:	learn: 0.2630546	total: 4.16s	remaining: 4.18s
499:	learn: 0.2629398	total: 4.17s	remaining: 4.17s
500:	learn: 0.2628646	total: 4.18s	remaining: 4.16s
501:	learn: 0.2628011	total: 4.18s	remaining: 4.15s
502:	learn: 0.2626937	total: 4.19s	remaining: 4.14s
503:	learn: 0.2626040	total: 4.2s	remaining: 4.13s
504:	learn: 0.2624724	total: 4.2s	remaining: 4.12s
505:	learn: 0.2623739	total: 4.21s	remaining: 4.11s
506:	learn: 0.2621872	total: 4.22s	remaining: 4.1s
507:	learn: 0.2620617	total: 4.22s	remaining: 4.09s
508:	learn: 0.2619942	total: 4.23s	remaining: 4.08s
509:	learn: 0.2618420	total: 4.24s	remaining: 4.07s
510:	learn: 0.2617180	total: 4.24s	remaining: 4.06s
511:	learn: 0.2615844	total: 4.25s	remaining: 4.05s
512:	learn: 0.2614523	total: 4.25s	remaining: 4.04s
513:	learn: 0.2613039	total: 4.26s	remaining: 4.03s
514:	learn: 0.2612237	total: 4.27s	remaining: 4.02s
515:	learn: 0.2611281	total: 4.28s	remaining: 4.01s
516:	learn: 0.2

662:	learn: 0.2459957	total: 5.4s	remaining: 2.74s
663:	learn: 0.2458794	total: 5.41s	remaining: 2.74s
664:	learn: 0.2458025	total: 5.41s	remaining: 2.73s
665:	learn: 0.2457366	total: 5.42s	remaining: 2.72s
666:	learn: 0.2456629	total: 5.43s	remaining: 2.71s
667:	learn: 0.2455723	total: 5.43s	remaining: 2.7s
668:	learn: 0.2454674	total: 5.44s	remaining: 2.69s
669:	learn: 0.2454545	total: 5.45s	remaining: 2.68s
670:	learn: 0.2453429	total: 5.45s	remaining: 2.67s
671:	learn: 0.2452502	total: 5.46s	remaining: 2.66s
672:	learn: 0.2451605	total: 5.46s	remaining: 2.65s
673:	learn: 0.2450282	total: 5.47s	remaining: 2.65s
674:	learn: 0.2449230	total: 5.48s	remaining: 2.64s
675:	learn: 0.2448484	total: 5.48s	remaining: 2.63s
676:	learn: 0.2447715	total: 5.49s	remaining: 2.62s
677:	learn: 0.2446916	total: 5.5s	remaining: 2.61s
678:	learn: 0.2445823	total: 5.5s	remaining: 2.6s
679:	learn: 0.2444618	total: 5.51s	remaining: 2.59s
680:	learn: 0.2443883	total: 5.51s	remaining: 2.58s
681:	learn: 0.244

...

277:	learn: 0.2886521	total: 2.75s	remaining: 7.14s
278:	learn: 0.2885437	total: 2.76s	remaining: 7.13s
279:	learn: 0.2883418	total: 2.76s	remaining: 7.11s
280:	learn: 0.2881649	total: 2.77s	remaining: 7.09s
281:	learn: 0.2879844	total: 2.78s	remaining: 7.07s
282:	learn: 0.2878288	total: 2.79s	remaining: 7.06s
283:	learn: 0.2876752	total: 2.79s	remaining: 7.04s
284:	learn: 0.2875127	total: 2.8s	remaining: 7.03s
285:	learn: 0.2873521	total: 2.81s	remaining: 7.01s
286:	learn: 0.2872249	total: 2.81s	remaining: 6.99s
287:	learn: 0.2871062	total: 2.82s	remaining: 6.97s
288:	learn: 0.2869866	total: 2.83s	remaining: 6.95s
289:	learn: 0.2868745	total: 2.83s	remaining: 6.93s
290:	learn: 0.2867145	total: 2.84s	remaining: 6.92s
291:	learn: 0.2865661	total: 2.85s	remaining: 6.9s
292:	learn: 0.2863958	total: 2.85s	remaining: 6.88s
293:	learn: 0.2861569	total: 2.86s	remaining: 6.86s
294:	learn: 0.2859855	total: 2.86s	remaining: 6.84s
295:	learn: 0.2858382	total: 2.87s	remaining: 6.83s
296:	learn: 0.

448:	learn: 0.2650764	total: 3.98s	remaining: 4.89s
449:	learn: 0.2650252	total: 3.99s	remaining: 4.88s
450:	learn: 0.2648400	total: 4s	remaining: 4.87s
451:	learn: 0.2646798	total: 4s	remaining: 4.86s
452:	learn: 0.2645535	total: 4.01s	remaining: 4.84s
453:	learn: 0.2644505	total: 4.02s	remaining: 4.83s
454:	learn: 0.2642034	total: 4.02s	remaining: 4.82s
455:	learn: 0.2641117	total: 4.03s	remaining: 4.81s
456:	learn: 0.2640147	total: 4.04s	remaining: 4.8s
457:	learn: 0.2638871	total: 4.05s	remaining: 4.79s
458:	learn: 0.2637614	total: 4.06s	remaining: 4.78s
459:	learn: 0.2636110	total: 4.06s	remaining: 4.77s
460:	learn: 0.2635267	total: 4.07s	remaining: 4.76s
461:	learn: 0.2634056	total: 4.08s	remaining: 4.75s
462:	learn: 0.2632625	total: 4.09s	remaining: 4.74s
463:	learn: 0.2630884	total: 4.09s	remaining: 4.73s
464:	learn: 0.2629671	total: 4.1s	remaining: 4.72s
465:	learn: 0.2628204	total: 4.11s	remaining: 4.7s
466:	learn: 0.2626324	total: 4.11s	remaining: 4.69s
467:	learn: 0.2624756

610:	learn: 0.2464745	total: 5.22s	remaining: 3.32s
611:	learn: 0.2463160	total: 5.23s	remaining: 3.31s
612:	learn: 0.2461496	total: 5.24s	remaining: 3.31s
613:	learn: 0.2460750	total: 5.24s	remaining: 3.3s
614:	learn: 0.2459319	total: 5.25s	remaining: 3.29s
615:	learn: 0.2458322	total: 5.26s	remaining: 3.28s
616:	learn: 0.2457063	total: 5.27s	remaining: 3.27s
617:	learn: 0.2456446	total: 5.28s	remaining: 3.27s
618:	learn: 0.2455671	total: 5.3s	remaining: 3.26s
619:	learn: 0.2455399	total: 5.32s	remaining: 3.26s
620:	learn: 0.2454514	total: 5.33s	remaining: 3.25s
621:	learn: 0.2453404	total: 5.33s	remaining: 3.24s
622:	learn: 0.2452578	total: 5.34s	remaining: 3.23s
623:	learn: 0.2451656	total: 5.34s	remaining: 3.22s
624:	learn: 0.2450328	total: 5.35s	remaining: 3.21s
625:	learn: 0.2448886	total: 5.36s	remaining: 3.2s
626:	learn: 0.2448010	total: 5.36s	remaining: 3.19s
627:	learn: 0.2447292	total: 5.37s	remaining: 3.18s
628:	learn: 0.2445765	total: 5.38s	remaining: 3.17s
629:	learn: 0.2

780:	learn: 0.2307426	total: 6.46s	remaining: 1.81s
781:	learn: 0.2306672	total: 6.47s	remaining: 1.8s
782:	learn: 0.2305633	total: 6.48s	remaining: 1.79s
783:	learn: 0.2304732	total: 6.49s	remaining: 1.79s
784:	learn: 0.2303890	total: 6.49s	remaining: 1.78s
785:	learn: 0.2303633	total: 6.5s	remaining: 1.77s
786:	learn: 0.2303049	total: 6.51s	remaining: 1.76s
787:	learn: 0.2302380	total: 6.51s	remaining: 1.75s
788:	learn: 0.2301894	total: 6.52s	remaining: 1.74s
789:	learn: 0.2301332	total: 6.53s	remaining: 1.74s
790:	learn: 0.2300882	total: 6.53s	remaining: 1.73s
791:	learn: 0.2299966	total: 6.54s	remaining: 1.72s
792:	learn: 0.2299136	total: 6.55s	remaining: 1.71s
793:	learn: 0.2298299	total: 6.55s	remaining: 1.7s
794:	learn: 0.2297305	total: 6.56s	remaining: 1.69s
795:	learn: 0.2296258	total: 6.57s	remaining: 1.68s
796:	learn: 0.2295432	total: 6.57s	remaining: 1.67s
797:	learn: 0.2294910	total: 6.58s	remaining: 1.67s
798:	learn: 0.2294315	total: 6.58s	remaining: 1.66s
799:	learn: 0.2

948:	learn: 0.2188954	total: 7.7s	remaining: 414ms
949:	learn: 0.2188349	total: 7.71s	remaining: 406ms
950:	learn: 0.2187457	total: 7.71s	remaining: 397ms
951:	learn: 0.2186994	total: 7.72s	remaining: 389ms
952:	learn: 0.2185636	total: 7.73s	remaining: 381ms
953:	learn: 0.2185150	total: 7.73s	remaining: 373ms
954:	learn: 0.2184625	total: 7.74s	remaining: 365ms
955:	learn: 0.2183669	total: 7.75s	remaining: 357ms
956:	learn: 0.2182506	total: 7.75s	remaining: 348ms
957:	learn: 0.2181629	total: 7.76s	remaining: 340ms
958:	learn: 0.2181050	total: 7.77s	remaining: 332ms
959:	learn: 0.2180620	total: 7.77s	remaining: 324ms
960:	learn: 0.2179556	total: 7.78s	remaining: 316ms
961:	learn: 0.2178796	total: 7.79s	remaining: 308ms
962:	learn: 0.2178195	total: 7.79s	remaining: 299ms
963:	learn: 0.2177401	total: 7.8s	remaining: 291ms
964:	learn: 0.2176497	total: 7.81s	remaining: 283ms
965:	learn: 0.2175620	total: 7.81s	remaining: 275ms
966:	learn: 0.2174933	total: 7.82s	remaining: 267ms
967:	learn: 0.

In [148]:



scores=pd.DataFrame(scores)
scores

,0
0,0.896396
1,0.885886
2,0.877628
3,0.885886
4,0.874530
5,0.882795
6,0.866266
7,0.874530
8,0.887303
9,0.873028


In [109]:
models

{'model1': <catboost.core.CatBoostClassifier at 0x1e2d1c1dd60>,
 'model2': <catboost.core.CatBoostClassifier at 0x1e2d1c1df70>,
 'model3': <catboost.core.CatBoostClassifier at 0x1e2d161a7f0>,
 'model4': <catboost.core.CatBoostClassifier at 0x1e2d1c1d430>,
 'model5': <catboost.core.CatBoostClassifier at 0x1e2d1c1d700>,
 'model6': <catboost.core.CatBoostClassifier at 0x1e2cbdfd9a0>,
 'model7': <catboost.core.CatBoostClassifier at 0x1e2d1c1d040>,
 'model8': <catboost.core.CatBoostClassifier at 0x1e2d1c1dee0>,
 'model9': <catboost.core.CatBoostClassifier at 0x1e2d179ddc0>,
 'model10': <catboost.core.CatBoostClassifier at 0x1e2d179d820>}

In [110]:
(19020-5706)-5706

7608

In [66]:
models_cat=models.copy()
X_tests = X[10000:13314]
y_tests = y[10000:13314]
y_tests=pd.DataFrame(y_tests,columns=["Class"])
y_tests.loc[y_tests['Class'] == "a", 'Class',] = 0
y_tests.loc[y_tests['Class'] == "b", 'Class',] = 1
y_tests=y_tests.astype(int)
for i in models:
    pred=models[i].predict(X_tests)
    accuracy=accuracy_score(y_tests, pred)*100
    print(accuracy,model)

88.20156910078455 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
87.86964393482197 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
87.89981894990947 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
87.96016898008449 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
88.05069402534701 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
88.14121907060954 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
87.990343995172 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
88.17139408569705 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
88.0205190102595 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
87.83946891973446 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
87.92999396499698 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
87.83946891973446 <catboost.core.CatBoostClassifier object at 0x000001E2D181FBE0>
88.05069402534701 <

In [381]:
y_tests

,Class
10000,1
10001,0
10002,1
10003,0
10004,1
...,...
13309,1
13310,0
13311,0
13312,0


In [149]:
test_dataset_X = X[5706:]
test_dataset_y = y[5706:]
pd.DataFrame(test_dataset_y).isnull().sum()

Class    5706
dtype: int64

In [545]:
pred=models["model60"].predict(test_dataset_X)

KeyError: 'model60'

In [340]:
pred=pred.astype(int)

In [341]:
len(pred)

13314

In [224]:
X_test=X_test.drop(columns=["ID"])
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=[ 'Feature 10'])

X_test = encoder.fit_transform(X_test)

scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [92]:
X_test=X_test.drop(columns=["ID"])

In [93]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=[ 'Feature 10'])

X_test = encoder.fit_transform(X_test)

scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

In [111]:
X_test=X[13314:]

In [150]:
pred=models["model1"].predict(X_test)

In [151]:
importance=models["model1"].feature_importances_
importance

array([17.79087574, 19.34913414, 19.17751681, 10.32891288, 23.79668544,
        9.55687499])

In [130]:
 ['Feature 1', 'Feature 2', 'Feature 3', 'Feature 7',  'Feature 9', 'Feature 11']

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [126]:
importance[importance>7]

array([ 7.80334263,  8.82918552, 14.57525231, 15.2443597 , 16.2544056 ,
       21.4841755 ])

In [152]:
pred

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [153]:
pred=pd.DataFrame(pred)
pred

,0
0,0
1,0
2,1
3,1
4,1
...,...
5701,1
5702,1
5703,1
5704,0


In [154]:
test=pd.read_csv("test.csv")
y_pred=pd.concat([test["ID"],pred],axis=1)

In [155]:
y_pred.columns=["ID","Class"]

In [156]:
y_pred.loc[y_pred['Class'] == 0, 'Class',] = "a"
y_pred.loc[y_pred['Class'] == 1, 'Class',] = "b"



In [157]:
y_pred

,ID,Class
0,13154,a
1,2571,a
2,1320,b
3,16907,b
4,5570,b
...,...,...
5701,16632,b
5702,11032,b
5703,15068,b
5704,10456,a


In [158]:
y_pred.to_csv('extra5.csv', index = False)

In [ ]:
y_test=pd.read_csv("predict_CB.csv")

In [ ]:
y_test.loc[y_test['Class'] == "a", 'Class',] = int(0)
y_test.loc[y_test['Class'] == "b", 'Class',] = int(1)

In [ ]:
pred.values

In [349]:
y_test["Class"]=y_test["Class"].astype(int)
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test["Class"], pred)*100
accuracy

93.70837714686296

In [328]:
pred

,0
0,1
1,0
2,0
3,1
4,0
...,...
5701,1
5702,1
5703,1
5704,0


In [351]:
d=pd.read_csv('predictions_cross_val.csv')

In [352]:
d.to_csv('predictions_by.csv', index = False)